In [1]:
import time
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, Window
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,FloatType,DoubleType,LongType
from pyspark.sql.functions import to_timestamp, to_date, lit, sort_array, collect_list, col, udf
import pyspark.sql.functions as f
import json
from pyspark.sql.functions import udf
from pyspark.sql.functions import unix_timestamp, from_unixtime
from pyspark.sql.types import DateType
spark.conf.set("spark.sql.session.timeZone", "UTC+08:00")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1612666188924_0004,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#three days
#enter the address for data
#for feature
date = '20201221'

user_s3_address = 's3://douyu-datalab/three_days_full/tmp_recom_user_feature_1d_'+date+'.csv'
item_s3_address = 's3://douyu-datalab/three_days_full/tmp_recom_item_feature_1d_'+date+'.csv'
#for label
weight_s3_address = 's3://douyu-datalab/three_days_full/tmp_recom_weight_data_1d_'+date+'.csv'
action_s3_address = 's3://douyu-datalab/three_days_full/tmp_recom_action_code_1d_'+date+'.csv'
black_user_s3_address = 's3://douyu-datalab/three_days_full/tmp_recom_black_user_1d_'+date+'.csv'


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# load data

In [3]:
Timer1 = time.time()
df_item = spark.read.format('csv').\
    options(header='true', inferSchema='false', encoding='utf8', sep='\t').\
    load(item_s3_address)

item_schema = StructType([
    StructField('room_id', StringType(), True),
    StructField('room_feature', StringType(), True),
    StructField('anchor_level', StringType(), True),
    StructField('anchor_score', StringType(), True),
    StructField('is_top200_anchor', StringType(), True),
    StructField('room_click_num_1_day', StringType(), True),
    StructField('room_click_num_7_day', StringType(), True),
    StructField('room_click_num_15_day', StringType(), True),
    StructField('room_ctr_1_day', StringType(), True),
    StructField('room_ctr_7_day', StringType(), True),
    StructField('room_ctr_15_day', StringType(), True),
    StructField('i_hot_mon', StringType(), True),
    StructField('i_hot_fri', StringType(), True),
    StructField('i_hot_weekend', StringType(), True),
    StructField('i_hot_weekday', StringType(), True),
    StructField('i_ctr_mon', StringType(), True),
    StructField('i_ctr_fri', StringType(), True),
    StructField('i_ctr_weekend', StringType(), True),
    StructField('i_ctr_weekday', StringType(), True),
    StructField('room_period_favor_30d', StringType(), True),
    StructField('guess_cnt_1d', StringType(), True),
    StructField('guess_cnt_7d', StringType(), True),
    StructField('guess_cnt_15d', StringType(), True),
    StructField('raffle_cnt_1d', StringType(), True),
    StructField('raffle_cnt_7d', StringType(), True),
    StructField('raffle_cnt_15d', StringType(), True),
    StructField('room_watch_newmbr_cnt_30d', StringType(), True),
    StructField('room_effwatch_mbr_ratio_30d', StringType(), True),
    StructField('room_follow_mbr_ratio_30d', StringType(), True),
    StructField('room_click_repeat_uids_1d', StringType(), True),
    StructField('room_click_repeat_uids_7d', StringType(), True),
    StructField('room_click_repeat_uids_15d', StringType(), True),
    StructField('click_repeat_avg_nums_15d', StringType(), True),
    StructField('room_repeat_uid_rate_15d', StringType(), True),
]
)



df_item = spark.createDataFrame(data = df_item.rdd, schema = item_schema)

df_item.show()

#fillna
df_item = df_item.fillna(0)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+------------+------------+----------------+--------------------+--------------------+---------------------+--------------+--------------+---------------+---------+---------+-------------+-------------+---------+---------+-------------+-------------+---------------------+------------+------------+-------------+-------------+-------------+--------------+-------------------------+---------------------------+-------------------------+-------------------------+-------------------------+--------------------------+-------------------------+------------------------+
|             room_id|        room_feature|anchor_level|anchor_score|is_top200_anchor|room_click_num_1_day|room_click_num_7_day|room_click_num_15_day|room_ctr_1_day|room_ctr_7_day|room_ctr_15_day|i_hot_mon|i_hot_fri|i_hot_weekend|i_hot_weekday|i_ctr_mon|i_ctr_fri|i_ctr_weekend|i_ctr_weekday|room_period_favor_30d|guess_cnt_1d|guess_cnt_7d|guess_cnt_15d|raffle_cnt_1d|raffle_cnt_7d|raffle_cnt

In [4]:
df_user = spark.read.format('csv').\
    options(header='true', inferSchema='true', encoding='utf8', sep='\t').\
    load(user_s3_address)

user_schema = StructType([
    StructField('uid', StringType(), True),
StructField('own_room', StringType(), True),
StructField('email_status', StringType(), True),
StructField('phone_status', StringType(), True),
StructField('sex', StringType(), True),
StructField('src', StringType(), True),
StructField('city', StringType(), True),
StructField('province', StringType(), True),
StructField('brand', StringType(), True),
StructField('model', StringType(), True),
StructField('active', StringType(), True),
StructField('level', StringType(), True),
StructField('days', StringType(), True),
StructField('msg_cnt', StringType(), True),
StructField('effect_watch_days', StringType(), True),
StructField('effect_watch_room_cnt', StringType(), True),
StructField('watch_time', StringType(), True),
StructField('yuwan_tag', StringType(), True),
StructField('yuwan_cnt', StringType(), True),
StructField('yuchi_cnt', StringType(), True),
StructField('follownum', StringType(), True),
StructField('rich_tag', StringType(), True),
StructField('loser_tag', StringType(), True),
StructField('cate', StringType(), True),
StructField('cate_length', StringType(), True),
StructField('tag', StringType(), True),
StructField('tag_length', StringType(), True),
StructField('child', StringType(), True),
StructField('child_length', StringType(), True),
StructField('user_tag_favor_1_day', StringType(), True),
StructField('tag_ratio_1_day', StringType(), True),
StructField('user_tag_favor_7_day', StringType(), True),
StructField('tag_ratio_7_day', StringType(), True),
StructField('user_tag_favor_15_day', StringType(), True),
StructField('tag_ratio_15_day', StringType(), True),
StructField('u_active_mon', StringType(), True),
StructField('u_active_fri', StringType(), True),
StructField('u_active_weekday', StringType(), True),
StructField('u_active_weekend', StringType(), True),
StructField('u_favor_mon', StringType(), True),
StructField('u_favor_fri', StringType(), True),
StructField('u_favor_weekday', StringType(), True),
StructField('u_favor_weekend', StringType(), True),
StructField('u_i_c_favor_mon', StringType(), True),
StructField('u_i_c_favor_fri', StringType(), True),
StructField('u_i_c_favor_weekday', StringType(), True),
StructField('u_i_c_favor_weekend', StringType(), True),
StructField('follow_room_top10_m', StringType(), True),
StructField('unfollow_room_top10_m', StringType(), True),
StructField('follow_yuba_room_top10_m', StringType(), True),
StructField('favor_room_1d_top10_m', StringType(), True),
StructField('favor_room_7d_top10_m', StringType(), True),
StructField('near_login_days', StringType(), True),
StructField('near_watch_days', StringType(), True),
StructField('watch_period_favor_30d', StringType(), True),
StructField('guess_join_cnt_1d', StringType(), True),
StructField('guess_join_cnt_7d', StringType(), True),
StructField('guess_join_cnt_15d', StringType(), True),
StructField('raffle_join_cnt_1d', StringType(), True),
StructField('raffle_join_cnt_7d', StringType(), True),
StructField('raffle_join_cnt_15d', StringType(), True),
StructField('effect_watch_room_cnt_1d', StringType(), True),
StructField('effect_watch_room_cnt_7d', StringType(), True),
StructField('effect_watch_room_cnt_15d', StringType(), True),
StructField('effect_watch_live_cate_cnt_1d', StringType(), True),
StructField('effect_watch_live_cate_cnt_7d', StringType(), True),
StructField('effect_watch_live_cate_cnt_15d', StringType(), True),
StructField('effect_watch_cate_rooms_cnt_1d', StringType(), True),
StructField('effect_watch_cate_rooms_cnt_7d', StringType(), True),
StructField('effect_watch_cate_rooms_cnt_15d', StringType(), True),
StructField('uid_attend_cate_aids_nums', StringType(), True),
StructField('uid_attend_cate_aids_rate', StringType(), True),
StructField('uid_watch_cate_cnt_1d', StringType(), True),
StructField('uid_watch_cate_cnt_7d', StringType(), True),
StructField('uid_watch_cate_cnt_15d', StringType(), True),
StructField('uid_watch_cate_maxtime_15d', StringType(), True),
StructField('uid_watch_cate_mintime_15d', StringType(), True),
StructField('user_group', StringType(), True),
]
)


df_user = spark.createDataFrame(data = df_user.rdd, schema = user_schema)

df_user.show()

#fillna
df_user = df_user.fillna(0)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------+------------+------------+---+--------------+--------+--------+-------+-------------+------+-----+----+-------+-----------------+---------------------+----------+---------+---------+---------+---------+--------+---------+--------------------+-----------+--------------------+----------+--------------------+------------+--------------------+--------------------+--------------------+--------------------+---------------------+--------------------+------------+------------+----------------+----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+---------------------+------------------------+---------------------+---------------------+--------------------+---------------+----------------------+-----------------+-----------------+------------------+------------------+------------------+-------------------+----------

In [5]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,FloatType,DoubleType,LongType
# uid: 
# room_id: 
# watch_time: (ms)
# watch_cnt: click times per day
df_weight = spark.read.format('csv').\
    options(header='true', inferSchema='true', encoding='utf8', sep='\t').\
    load(weight_s3_address)

weight_schema = StructType([
    StructField('uid', StringType(), True),
    StructField('room_id', StringType(), True),
    StructField('watch_time', LongType(), True),
    StructField('watch_cnt', IntegerType(), True),
]
)


df_weight = spark.createDataFrame(data = df_weight.rdd, schema = weight_schema)

df_weight.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+----------+---------+
|                 uid|             room_id|watch_time|watch_cnt|
+--------------------+--------------------+----------+---------+
|d41d8cd98f00b204e...|0d02c763e487d0d0c...|       965|        1|
|d41d8cd98f00b204e...|aedbb569a96b299f6...|      5833|        1|
|d41d8cd98f00b204e...|db278c63f7701b7ef...|    500403|        5|
|d41d8cd98f00b204e...|b112d3732c00481dd...|    344882|        1|
|d41d8cd98f00b204e...|b46626d0b1d7c9dc5...| 232656130|      359|
|d41d8cd98f00b204e...|d752301454cd17aab...| 302142468|      698|
|d41d8cd98f00b204e...|ac5dee31fad471e0a...|  26551349|      270|
|d41d8cd98f00b204e...|b8955fbd59e3e7688...|    274390|        2|
|d41d8cd98f00b204e...|9de85d862bc86cce2...|   9572343|      108|
|d41d8cd98f00b204e...|95cfbc9160f4c2458...|   1568506|       11|
|d41d8cd98f00b204e...|5ed9d6f25cc4b49fb...|  24297507|       16|
|d41d8cd98f00b204e...|1f04dfccc3bb2a277...|   4521732|       21|
|d41d8cd98f00b204e...|b8a

In [6]:
# uid: 
# room_id: 
# action_code: show_recommend|click_recommend
# user_pos: position in push list(after sort?)
# ct_pt: platform(ios/android)
# src: recall source
df_action_code = spark.read.format('csv').\
    options(header='true', inferSchema='False', encoding='utf8', sep='\t').\
    load(action_s3_address)

action_schema = StructType([
    StructField('dateline', StringType(), True),
    StructField('uid', StringType(), True),
    StructField('room_id', StringType(), True),
    StructField('action_code', StringType(), True),
    StructField('user_pos', StringType(), True),
    StructField('ct_pt', StringType(), True),
    StructField('src', StringType(), True),
]
)

df_action_code = spark.createDataFrame(data = df_action_code.rdd, schema = action_schema)

df_action_code.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------------------+--------------------+--------------------+--------+-------+----+
|     dateline|                 uid|             room_id|         action_code|user_pos|  ct_pt| src|
+-------------+--------------------+--------------------+--------------------+--------+-------+----+
|1608520873257|3a0ebc104c8c6faf9...|38b3eff8baf566274...|show_recommend_re...|       1|    ios|9831|
|1608562029075|7f894553142e46cf3...|ae8b5aa26a3ae3161...|show_recommend_re...|       3|    ios|1005|
|1608565452016|8d9fc5944d88166c1...|6d70d6bd21e71385f...|show_recommend_re...|      59|android|   0|
|1608565345690|8d9fc5944d88166c1...|3ed304a448828ee2c...|show_recommend_re...|     172|android|1004|
|1608518783435|b57629c6aaffeaed1...|ae9fb56eaa2096c74...|show_recommend_re...|       3|android|1005|
|1608531578847|ac6ddf1e3c74ca94c...|89209f444391e9c72...|click_recommend_r...|       2|    ios|  55|
|1608484996710|69c8bd124b713e8b9...|0a18cf521deddea8c...|show_recommend_re...|     114|    

In [7]:
df_black_user = spark.read.format('csv').\
    options(header='true', inferSchema='true', encoding='utf8', sep='\t').\
    load(black_user_s3_address)

black_user_schema = StructType([
    StructField('uid', StringType(), True)
]
)

df_black_user = spark.createDataFrame(data = df_black_user.rdd, schema = black_user_schema)

df_black_user.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|                 uid|
+--------------------+
|5bb3e97dd0adecd7e...|
|1a36f61185bf6ff40...|
|5f0d7801c1ec95405...|
|20f36cd5dd3d04450...|
|c7aefd8b11c353e88...|
|03ae395301b6a6c33...|
|da3708694112c94dc...|
|dba2d22b83fffcfe3...|
|60614522f3f94a04e...|
|aaa7c2a8ed4f06160...|
|7fe449ffcf4aaec8f...|
|34416d1ec348fbf2f...|
|2ded47296a2b9e43f...|
|2e4e8879b24ba9a9b...|
|f8dcc9796922bf36a...|
|79225f4a63ce1bf97...|
|82e7d0e6ac4f1fd70...|
|c490411d314bbfef5...|
|669604f3b56ee34ad...|
|b60feec01eddbf2c9...|
+--------------------+
only showing top 20 rows

# process label

In [8]:
#convert to datetime
df_action_code = df_action_code.withColumn('datetamp', (df_action_code.dateline/1e3).cast(IntegerType()))
df_action_code = df_action_code.withColumn('datetamp2', from_unixtime(df_action_code.datetamp))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
from pyspark.sql import functions as F

df_action_code.groupBy("action_code").agg(F.countDistinct("uid", "room_id","dateline")).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------+--------------------------------------+
|action_code                   |count(DISTINCT uid, room_id, dateline)|
+------------------------------+--------------------------------------+
|click_recommend_recom_hot_room|10613801                              |
|show_recommend_recom_liveroom |766                                   |
|click_recommend_recom_liveroom|36                                    |
|show_recommend_recom_hot_room |80878007                              |
+------------------------------+--------------------------------------+

In [10]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,FloatType,DoubleType,LongType

def convert_label(x):
    if x=='click_recommend_recom_hot_room' or x=='click_recommend_recom_liveroom':
        return 1
    else:
        return 0

#use age, owner_level,sex,fans
udf_func_convert = udf(lambda x: convert_label(x),returnType=IntegerType())
df_action_code = df_action_code.withColumn("label",udf_func_convert(df_action_code["action_code"]))

df_action_code_agg = df_action_code.groupby(df_action_code.uid,df_action_code.room_id).\
agg(f.max('label').alias("label_click"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
df_action_code.groupBy("label").agg(F.countDistinct("uid", "room_id")).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+----------------------------+
|label|count(DISTINCT uid, room_id)|
+-----+----------------------------+
|1    |10303567                    |
|0    |77854116                    |
+-----+----------------------------+

In [12]:
df_action_code.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- dateline: string (nullable = true)
 |-- uid: string (nullable = true)
 |-- room_id: string (nullable = true)
 |-- action_code: string (nullable = true)
 |-- user_pos: string (nullable = true)
 |-- ct_pt: string (nullable = true)
 |-- src: string (nullable = true)
 |-- datetamp: integer (nullable = true)
 |-- datetamp2: string (nullable = true)
 |-- label: integer (nullable = true)

In [13]:
df_action_code_agg.groupBy("label_click").agg(F.countDistinct("uid", "room_id")).show(truncate=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------------------------+
|label_click|count(DISTINCT uid, room_id)|
+-----------+----------------------------+
|1          |10303567                    |
|0          |67721205                    |
+-----------+----------------------------+

In [14]:
#group by userid, roomid
#此处将曝光未点击的记录，直接当成曝光点击的前置部分？？？如果两者时间差别比较大呢？
df_action_code_agg = df_action_code.groupby(df_action_code.uid,df_action_code.room_id).\
agg(f.max('label').alias("label_click"))
#left join watch to filter conversion
df_res = df_action_code_agg.join(df_weight,on=["room_id","uid"],how='left')
#convert watchtime to second
df_res2 = df_res.withColumn("time_min", df_res["watch_time"]/60000)

df_res2 = df_res2.fillna(0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
def convert_cvr_label(label_click,time_min):
    if int(label_click)==1 and float(time_min)>5:
        return 1
    else:
        return 0

    
udf_func_convert = udf(lambda x,y: convert_cvr_label(x,y),returnType=IntegerType())
df_res2 = df_res2.withColumn("cvr_label",udf_func_convert(df_res2["label_click"],df_res2["time_min"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
def convert_ctr_label(x,y):
    if int(x)==1 and float(y)>2:
        return 1
    else:
        return 0

udf_func_convert = udf(lambda x,y: convert_ctr_label(x,y),returnType=IntegerType())
df_res2 = df_res2.withColumn("ctr_label",udf_func_convert(df_res2["label_click"],df_res2["time_min"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
#filter out black user
df_black_user = df_black_user.withColumn("black_label",lit(1))
#join
df_res3 = df_res2.join(df_black_user,on=["uid"],how='left')
#filter 
df_res3 = df_res3.filter('black_label is null')
df_res3.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-----------+----------+---------+--------------------+---------+---------+-----------+
|                 uid|             room_id|label_click|watch_time|watch_cnt|            time_min|cvr_label|ctr_label|black_label|
+--------------------+--------------------+-----------+----------+---------+--------------------+---------+---------+-----------+
|000025bbc8e3fd2a3...|b56a18e0eacdf51aa...|          0|         0|        0|                 0.0|        0|        0|       null|
|000026c67a83fa72a...|f3475ed497caeab39...|          0|         0|        0|                 0.0|        0|        0|       null|
|000048f2d4a90b363...|89209f444391e9c72...|          0|    611792|       10|  10.196533333333333|        0|        0|       null|
|000069059776e2d3e...|5ec5fa5a5d586714f...|          0|         0|        0|                 0.0|        0|        0|       null|
|000073176e737404d...|72204d87e2fec8e0e...|          0|      2017|        1| 0.03361666666

In [18]:
# distribution
from pyspark.sql import functions as F

df_res3.groupBy("cvr_label").agg(F.countDistinct("uid", "room_id")).show()
        
# negative sampling 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------------------------+
|cvr_label|count(DISTINCT uid, room_id)|
+---------+----------------------------+
|        1|                     3712019|
|        0|                    74310081|
+---------+----------------------------+

In [19]:
# distribution
from pyspark.sql import functions as F

df_res3.groupBy("ctr_label").agg(F.countDistinct("uid", "room_id")).show()
        

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------------------------+
|ctr_label|count(DISTINCT uid, room_id)|
+---------+----------------------------+
|        1|                     4912055|
|        0|                    73110045|
+---------+----------------------------+

In [20]:
from pyspark.sql.functions import col

# negative sampling 
train_0= df_res3.where(col('ctr_label')==0).sample(True, 0.07, seed = 2018)
#step 2. Merge this data with label = 0 data

train_1=df_res3.where(col('ctr_label')==1)
train_final = train_0.union(train_1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
train_final.groupBy("ctr_label").agg(F.countDistinct("uid", "room_id")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------------------------+
|ctr_label|count(DISTINCT uid, room_id)|
+---------+----------------------------+
|        1|                     4912055|
|        0|                     4943557|
+---------+----------------------------+

In [22]:
train_final.groupBy("cvr_label").agg(F.countDistinct("uid", "room_id")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+----------------------------+
|cvr_label|count(DISTINCT uid, room_id)|
+---------+----------------------------+
|        1|                     3712019|
|        0|                     6143593|
+---------+----------------------------+

# process item 

In [23]:
#processs item feature
#cate_id
#child_id 
#tag_id
def regex_filter(x):
    try:
        a = list(json.loads(x).values())
    except:
        raise ("exception!")
    
    #return ['owner_exp','owner_uid','cate_id','age','sex','room_name',\
    #'owner_level','fans','child_id','cq','msgWords','tag_id','constellation']
    
    return a


#use age, owner_level,sex,fans
udf_func_age = udf(lambda x: regex_filter(x)[3],returnType=IntegerType())
udf_func_owner_level = udf(lambda x: regex_filter(x)[6],returnType=IntegerType())
udf_func_sex = udf(lambda x: regex_filter(x)[4],returnType=StringType())
udf_func_fans = udf(lambda x: regex_filter(x)[7],returnType=IntegerType())
udf_func_cateid = udf(lambda x: regex_filter(x)[2],returnType=StringType())
udf_func_childid = udf(lambda x: regex_filter(x)[8],returnType=StringType())
udf_func_tagid = udf(lambda x: regex_filter(x)[11],returnType=StringType())
udf_func_constellation = udf(lambda x: regex_filter(x)[12],returnType=StringType())

df_item = df_item.withColumn("item_age",udf_func_age(df_item["room_feature"])).\
withColumn("owner_level",udf_func_owner_level(df_item["room_feature"])).\
withColumn("item_sex",udf_func_sex(df_item["room_feature"])).\
withColumn("fans",udf_func_fans(df_item["room_feature"])).\
withColumn("cate_id",udf_func_cateid(df_item["room_feature"])).\
withColumn("child_id",udf_func_childid(df_item["room_feature"])).\
withColumn("tag_id",udf_func_tagid(df_item["room_feature"])).\
withColumn("constellation",udf_func_constellation(df_item["room_feature"]))

df_item = df_item.fillna(0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
#join result

columns_to_drop = ['room_feature']
df_item_res = df_item.drop(*columns_to_drop)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
df_item_res.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- room_id: string (nullable = true)
 |-- anchor_level: string (nullable = true)
 |-- anchor_score: string (nullable = true)
 |-- is_top200_anchor: string (nullable = true)
 |-- room_click_num_1_day: string (nullable = true)
 |-- room_click_num_7_day: string (nullable = true)
 |-- room_click_num_15_day: string (nullable = true)
 |-- room_ctr_1_day: string (nullable = true)
 |-- room_ctr_7_day: string (nullable = true)
 |-- room_ctr_15_day: string (nullable = true)
 |-- i_hot_mon: string (nullable = true)
 |-- i_hot_fri: string (nullable = true)
 |-- i_hot_weekend: string (nullable = true)
 |-- i_hot_weekday: string (nullable = true)
 |-- i_ctr_mon: string (nullable = true)
 |-- i_ctr_fri: string (nullable = true)
 |-- i_ctr_weekend: string (nullable = true)
 |-- i_ctr_weekday: string (nullable = true)
 |-- room_period_favor_30d: string (nullable = true)
 |-- guess_cnt_1d: string (nullable = true)
 |-- guess_cnt_7d: string (nullable = true)
 |-- guess_cnt_15d: string (nullable = 

# process user

In [26]:
#join result
#need to do embedding: uid/src/province/city/brand
#df_user_res = df_user.select(['uid','own_room','email_status','phone_status','sex','province'\
#                             'city','brand','model','active','level','days'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# join results 

In [27]:
#join result
#全量

df_weight_item = df_item_res.join(df_res3.select(["uid","room_id","cvr_label","ctr_label"]), on="room_id", how="right")
df_res_all = df_weight_item.join(df_user,on='uid',how='left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
#join result
#采样

df_weight_item_sample = df_item_res.join(train_final.select(["uid","room_id","cvr_label","ctr_label"]), on="room_id", how="right")
df_res_all_sample = df_weight_item_sample.join(df_user,on='uid',how='left')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# save results 

In [29]:
#save results
#df_train, df_test = df_res_all.randomSplit([0.9, 0.1])
Timer1 = time.time()

save_name = 's3://douyu-datalab/train_{}_gdate0207_full.csv'.format(date)

df_res_all.write.format('csv').\
    option('header', True).mode('overwrite').option('sep', '\t').\
    save(save_name)

print("It take {:.2f} mimutes to finish".format((time.time()-Timer1)/60))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

It take 12.53 mimutes to finish

In [30]:
#save results
#df_train, df_test = df_res_all.randomSplit([0.9, 0.1])

Timer1 = time.time()

save_name = 's3://douyu-datalab/train_{}_gdate0207_sample.csv'.format(date)

df_res_all_sample.write.format('csv').\
    option('header', True).mode('overwrite').option('sep', '\t').\
    save(save_name)

print("It take {:.2f} mimutes to finish".format((time.time()-Timer1)/60))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

It take 8.64 mimutes to finish